In [1]:
from datetime import date

last_coupon = date(2023, 2, 28)
next_coupon = date(2023, 8, 31)
valuation_date = date(2023, 4, 18)
forward_date = date(2023, 8, 1)

face = 100
coupon_rate = 0.04
coupon_payment = face * coupon_rate / 2

days_in_period = (next_coupon - last_coupon).days
days_to_valuation = (valuation_date - last_coupon).days
days_to_forward = (forward_date - last_coupon).days

ai_t = coupon_payment * days_to_valuation / days_in_period
ai_T = coupon_payment * days_to_forward / days_in_period

days_in_period, days_to_valuation, days_to_forward, ai_t, ai_T


(184, 49, 154, 0.532608695652174, 1.673913043478261)

In [4]:
from datetime import date

last_coupon = date(2023, 2, 28)
next_coupon = date(2023, 8, 31)
t = date(2023, 4, 18)
T = date(2023, 8, 1)

face = 100
coupon_rate = 0.04
repo_rate = 0.0485
S_clean = 102 + 2/32

C = face * coupon_rate / 2

days_period = (next_coupon - last_coupon).days
days_t = (t - last_coupon).days
days_T = (T - last_coupon).days
tau = (T - t).days / 360

AI_t = C * days_t / days_period
AI_T = C * days_T / days_period

dirty_t = S_clean + AI_t
F_clean = dirty_t * (1 + repo_rate * tau) - AI_T

whole = int(F_clean)
ths = round((F_clean - whole) * 32)

dirty_t, tau, F_clean, f"{whole}-{ths:02d}"


(102.59510869565217, 0.2916666666666667, 102.37248896059782, '102-12')

In [5]:
spot = S_clean
forward = F_clean

forward_drop = spot - forward

forward_drop_32nds = forward_drop * 32

spot, forward, round(forward_drop, 4), forward_drop_32nds


(102.0625, 102.37248896059782, -0.31, -9.919646739130258)

The forward price is higher than the spot price (the forward drop is negative).

This occurs because the bond has positive carry over the forward horizon:
the coupon accrual earned between today and the forward date exceeds the financing cost implied by the repo rate. When carry is positive, the forward price is bid above the spot price.

In [6]:
coupon_income = face * coupon_rate * tau

financing_cost = dirty_t * repo_rate * tau

carry = coupon_income - financing_cost

carry_32nds = carry * 32

round(coupon_income, 4), round(financing_cost, 4), round(carry, 4), carry_32nds


(1.1667, 1.4513, -0.2846, -9.108052536231888)

In [7]:
difference = forward_drop - carry

forward_drop, carry, difference


(-0.30998896059782055, -0.2846266417572465, -0.02536231884057405)

The forward drop and the carry are approximately, but not exactly, equal because they are computed using slightly different conventions and approximations.

In particular:

The forward price is based on financing the dirty price and then subtracting future accrued interest, which embeds exact compounding on the full dirty price.

The carry calculation approximates coupon income using a simple pro-rated annual coupon and financing cost using simple interest.

Differences in day-count conventions (ACT/ACT for accrued interest vs ACT/360 for repo and carry) also introduce a small mismatch.

In [8]:
from datetime import date

t = date(2023, 4, 18)
coupon_date = date(2023, 8, 31)
T = date(2023, 10, 15)
last_coupon = date(2023, 2, 28)
next_coupon = coupon_date

face = 100
coupon_rate = 0.04
repo_rate = 0.0485
S_clean = 102 + 2/32

C = face * coupon_rate / 2

days_period = (next_coupon - last_coupon).days
days_t = (t - last_coupon).days
days_T = (T - next_coupon).days

AI_t = C * days_t / days_period
AI_T = C * days_T / days_period

dirty_t = S_clean + AI_t

tau_pre = (coupon_date - t).days / 360
tau_post = (T - coupon_date).days / 360

value_at_coupon = dirty_t * (1 + repo_rate * tau_pre)
value_after_coupon = value_at_coupon - C

value_at_T = value_after_coupon * (1 + repo_rate * tau_post)

F_clean = value_at_T - AI_T

tau_total = (T - t).days / 360
F_no_coupon_adj = dirty_t * (1 + repo_rate * tau_total) - AI_T

(
    (coupon_date - t).days, tau_pre,
    (T - coupon_date).days, tau_post,
    round(value_at_coupon, 4),
    round(value_after_coupon, 4),
    round(value_at_T, 4),
    round(AI_T, 4),
    round(F_clean, 4),
    round(F_no_coupon_adj, 4),
    round(F_no_coupon_adj - F_clean, 4)
)


(135,
 0.375,
 45,
 0.125,
 104.4611,
 102.4611,
 103.0822,
 0.4891,
 102.5931,
 104.5939,
 2.0008)

The forward price with an interim coupon is lower than the hypothetical no-coupon-adjustment forward price, and the difference is approximately the present value of the coupon(s) paid before delivery. This aligns with economic intuition: forward buyers only pay for what they will actually receive.

In [11]:
futures_price = 101.50

forward_price = forward

diff = forward_price - futures_price

forward_price, futures_price, round(diff, 4)

(102.37248896059782, 101.5, 0.8725)

Treasury futures prices are typically lower than the forward price because of the daily settlement mechanism:

Forward contracts are settled only at maturity, so there is no cash flow until delivery. The buyer carries the full financing cost over the life of the contract.

Futures contracts are marked to market daily, meaning gains and losses are settled every day.

Positive cash flows received earlier reduce the effective financing cost.

Negative cash flows paid earlier increase the cost, but on average, the convexity adjustment tends to lower the futures price relative to the forward price.

Thus, the daily settlement of futures transfers risk and cash flows over time, making the futures price slightly lower than the equivalent forward price.

In [13]:
Z = 0.986051
notional_forward = 10_000_000
futures_notional = 100_000

N_naive = notional_forward / futures_notional

N_tailed = N_naive * Z

N_trade = round(N_tailed)

N_naive, N_tailed, N_trade


(100.0, 98.60510000000001, 99)